<a href="https://colab.research.google.com/github/amontenegrot/COVID19/blob/main/COVID_19_%7C_Colombia.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Importar datos

Datos oficiales:
[Mundo](https://ourworldindata.org/coronavirus-source-data)

[Colombia](https://www.ins.gov.co/Noticias/Paginas/Coronavirus.aspx)

[Bogotá](http://saludata.saludcapital.gov.co/osb/index.php/datos-de-salud/enfermedades-trasmisibles/covid19/?unapproved=410&moderation-hash=b01afdf1c01305df1f6b8a6a02826da5#comment-410)

---
[Genética UNAL](http://www.genetica.unal.edu.co/gibbs/index.php?id=covid19-en-colombia)

In [ ]:
import pandas as pd
import numpy as np
import cufflinks as cf
#import pycountry

import plotly.express as px
import plotly.graph_objects as go
import plotly.offline as py
from plotly.subplots import make_subplots
from pylab import *
from scipy.optimize import curve_fit
from scipy.integrate import odeint

dfGlobal = pd.read_csv('https://covid.ourworldindata.org/data/ecdc/total_cases.csv')
dfGlobalMuertes = pd.read_csv('https://covid.ourworldindata.org/data/ecdc/total_deaths.csv')
dfCol = pd.read_csv('https://www.datos.gov.co/api/views/gt2j-8ykr/rows.csv')

In [ ]:
def Mayus(cadena):
  if cadena == 'Bogotá D.C.':
    return str(cadena)
  elif cadena == 'Cartagena D.T. y C.':
    return str(cadena)
  elif cadena == 'Barranquilla D.E.':
    return str(cadena)
  elif cadena == 'Santa Marta D.T. y C.':
    return str(cadena)
  elif cadena == 'Buenaventura D.E.':
    return str(cadena)
  elif cadena == 'Santa marta D.T. y C.':
    return str(cadena)
  elif cadena == 'Archipiélago de San Andrés Providencia y Santa Catalina':
    return 'San Andrés, Providencia y Santa Catalina'
  elif cadena == 'Hospital UCI':
    return str(cadena)
  #elif cadena == 'F':
    #return 'Mujer'
  #elif cadena == 'M':
    #return 'Hombre'
  else:
    str(cadena)
    return cadena.capitalize()

#se busca en una cadena y lo que se busca se reemplaza
def Reemplazar(cadena, buscar, reemplazo):
  buscar = str(buscar)
  reemplazo = str(reemplazo)
  return cadena.replace(buscar, reemplazo)

#Preprocesamiento

In [ ]:
dfCol.dtypes

ID de caso                   int64
Fecha de notificación       object
Codigo DIVIPOLA              int64
Ciudad de ubicación         object
Departamento o Distrito     object
atención                    object
Edad                         int64
Sexo                        object
Tipo                        object
Estado                      object
País de procedencia         object
FIS                         object
Fecha de muerte             object
Fecha diagnostico           object
Fecha recuperado            object
fecha reporte web           object
dtype: object

In [ ]:
dfCol.columns

Index(['ID de caso', 'Fecha de notificación', 'Codigo DIVIPOLA',
       'Ciudad de ubicación', 'Departamento o Distrito ', 'atención', 'Edad',
       'Sexo', 'Tipo', 'Estado', 'País de procedencia', 'FIS',
       'Fecha de muerte', 'Fecha diagnostico', 'Fecha recuperado',
       'fecha reporte web'],
      dtype='object')

In [ ]:
dfCol.tail()

,ID de caso,Fecha de notificación,Codigo DIVIPOLA,Ciudad de ubicación,Departamento o Distrito,atención,Edad,Sexo,Tipo,Estado,País de procedencia,FIS,Fecha de muerte,Fecha diagnostico,Fecha recuperado,fecha reporte web
7280,7281,2020-04-29T00:00:00.000,11001,Bogotá D.C.,Bogotá D.C.,Casa,34,F,En estudio,Leve,Colombia,2020-04-25T00:00:00.000,- -,2020-05-01T00:00:00.000,- -,2020-05-02T00:00:00.000
7281,7282,2020-04-28T00:00:00.000,11001,Bogotá D.C.,Bogotá D.C.,Casa,86,M,En estudio,Leve,Colombia,2020-04-28T00:00:00.000,- -,2020-05-01T00:00:00.000,- -,2020-05-02T00:00:00.000
7282,7283,2020-04-29T00:00:00.000,11001,Bogotá D.C.,Bogotá D.C.,Casa,3,F,En estudio,Leve,Colombia,2020-04-28T00:00:00.000,- -,2020-05-01T00:00:00.000,- -,2020-05-02T00:00:00.000
7283,7284,2020-05-01T00:00:00.000,11001,Bogotá D.C.,Bogotá D.C.,Casa,30,M,En estudio,Leve,Colombia,2020-04-26T00:00:00.000,- -,2020-05-01T00:00:00.000,- -,2020-05-02T00:00:00.000
7284,7285,2020-05-01T00:00:00.000,11001,Bogotá D.C.,Bogotá D.C.,Casa,53,M,Relacionado,Leve,Colombia,2020-05-01T00:00:00.000,- -,2020-05-01T00:00:00.000,- -,2020-05-02T00:00:00.000


In [ ]:
#Eliminar columnas innecesarias
dfCol = dfCol.drop(['ID de caso'], axis=1)
dfCol = dfCol.drop(['Codigo DIVIPOLA'], axis=1)
dfCol = dfCol.drop(['FIS'], axis=1)
dfCol = dfCol.drop(['Fecha de muerte'], axis=1)
dfCol = dfCol.drop(['Fecha diagnostico'], axis=1)
dfCol = dfCol.drop(['Fecha recuperado'], axis=1)
dfCol = dfCol.drop(['fecha reporte web'], axis=1)

In [ ]:
dfCol.columns

Index(['Fecha de notificación', 'Ciudad de ubicación',
       'Departamento o Distrito ', 'atención', 'Edad', 'Sexo', 'Tipo',
       'Estado', 'País de procedencia'],
      dtype='object')

In [ ]:
#Renombrar columnas
dfCol = dfCol.rename({'Fecha de notificación':'Fecha',
                      'Ciudad de ubicación':'Ciudad',
                      'atención':'Ubicacion',
                      'Departamento o Distrito ':'Departamento',
                      'País de procedencia':'Procedencia'}, axis=1)
#Nueva columna con número de casos
dfCol['Casos'] = 1

In [ ]:
dfCol.tail()

,Fecha,Ciudad,Departamento,Ubicacion,Edad,Sexo,Tipo,Estado,Procedencia,Casos
7280,2020-04-29T00:00:00.000,Bogotá D.C.,Bogotá D.C.,Casa,34,F,En estudio,Leve,Colombia,1
7281,2020-04-28T00:00:00.000,Bogotá D.C.,Bogotá D.C.,Casa,86,M,En estudio,Leve,Colombia,1
7282,2020-04-29T00:00:00.000,Bogotá D.C.,Bogotá D.C.,Casa,3,F,En estudio,Leve,Colombia,1
7283,2020-05-01T00:00:00.000,Bogotá D.C.,Bogotá D.C.,Casa,30,M,En estudio,Leve,Colombia,1
7284,2020-05-01T00:00:00.000,Bogotá D.C.,Bogotá D.C.,Casa,53,M,Relacionado,Leve,Colombia,1


In [ ]:
#Reemplazar errores de tipeo en cada columna (mayúscula inicial)
dfCol['Ciudad'] = dfCol['Ciudad'].apply(lambda x: Mayus(x))
dfCol['Departamento'] = dfCol['Departamento'].apply(lambda x: Mayus(x))
dfCol['Ubicacion'] = dfCol['Ubicacion'].apply(lambda x: str(x)) #Volver todos los valores tipo str
dfCol['Ubicacion'] = dfCol['Ubicacion'].apply(lambda x: Mayus(x))
dfCol['Sexo'] = dfCol['Sexo'].apply(lambda x: Mayus(x))
dfCol['Tipo'] = dfCol['Tipo'].apply(lambda x: Mayus(x))
dfCol['Estado'] = dfCol['Estado'].apply(lambda x: str(x)) #Volver todos los valores tipo str
dfCol['Estado'] = dfCol['Estado'].apply(lambda x: Mayus(x))

In [ ]:
dfCol['Estado'].value_counts()

Leve         6360
Moderado      475
Fallecido     324
Grave         122
Nan             4
Name: Estado, dtype: int64

In [ ]:
dfCol['Fecha'] = pd.to_datetime(dfCol['Fecha']) #Conversión a formato de 'fecha'

In [ ]:
dfCol.dtypes

Fecha           datetime64[ns]
Ciudad                  object
Departamento            object
Ubicacion               object
Edad                     int64
Sexo                    object
Tipo                    object
Estado                  object
Procedencia             object
Casos                    int64
dtype: object

In [ ]:
dfCol.head()

,Fecha,Ciudad,Departamento,Ubicacion,Edad,Sexo,Tipo,Estado,Procedencia,Casos
0,2020-03-02,Bogotá D.C.,Bogotá D.C.,Recuperado,19,F,Importado,Leve,Italia,1
1,2020-03-06,Guadalajara de buga,Valle del cauca,Recuperado,34,M,Importado,Leve,España,1
2,2020-03-07,Medellín,Antioquia,Recuperado,50,F,Importado,Leve,España,1
3,2020-03-09,Medellín,Antioquia,Recuperado,55,M,Relacionado,Leve,Colombia,1
4,2020-03-09,Medellín,Antioquia,Recuperado,25,M,Relacionado,Leve,Colombia,1


#Procesamiento

##Casos totales

In [ ]:
casos = dfCol['Casos'].sum()
print('Total de casos a nivel nacional:', casos)

Total de casos a nivel nacional: 7285


##Casos por sexo

In [ ]:
#Transformar el objeto 'value_counts' en DataFrame
dfSexo = dfCol['Sexo']
dfSexo = dfSexo.value_counts()
dfSexo = dfSexo.rename_axis('Sexo')
dfSexo = dfSexo.reset_index(name='Numero')

In [ ]:
dfSexo['Porcentaje'] = dfSexo['Numero'].apply(lambda x: (x/casos)*100)

dfSexo

,Sexo,Numero,Porcentaje
0,M,3924,53.864104
1,F,3361,46.135896


In [ ]:
fig = go.Figure(data=[go.Pie(labels=dfSexo['Sexo'], values=dfSexo['Numero'], hole=.3)])
fig.show()

##Casos por departamentos

In [ ]:
#Transformar el objeto 'value_counts' en DataFrame
dfDept = dfCol['Departamento']
dfDept = dfDept.value_counts()
dfDept = dfDept.rename_axis('Departamento')
dfDept = dfDept.reset_index(name='Numero')

In [ ]:
dfDept['Porcentaje'] = dfDept['Numero'].apply(lambda x: (x/casos)*100)

dfDept

,Departamento,Numero,Porcentaje
0,Bogotá D.C.,2846,39.066575
1,Valle del cauca,1021,14.015100
2,Meta,479,6.575154
3,Antioquia,477,6.547701
4,Cartagena D.T. y C.,309,4.241592
5,Cundinamarca,235,3.225806
6,Atlántico,213,2.923816
7,Risaralda,208,2.855182
8,Barranquilla D.E.,201,2.759094
9,Santa Marta D.T. y C.,176,2.415923


In [ ]:
fig = px.scatter(x=dfDept['Departamento'], y=dfDept['Numero'],
                 size=dfDept['Porcentaje'], #color=dfDept['Departamento'],
                 hover_name=dfDept['Departamento'], size_max=50,
                 title='Número de casos por departamento o distrito especial', 
                 labels=list(dfDept['Departamento']))
fig.show()

In [ ]:
fig = px.scatter(x=dfCol['Ciudad'], y=dfCol['Casos'],
                 size=dfDept['Porcentaje'], color=dfDept['Departamento'],
                 hover_name=dfDept['Departamento'], size_max=50,
                 title='Número de casos por departamento o distrito especial', 
                 labels=list(dfDept['Departamento']))
fig.show()

##Casos por ubicación

In [ ]:
#Transformar el objeto 'value_counts' en DataFrame
dfUbicacion = dfCol['Ubicacion']
dfUbicacion = dfUbicacion.value_counts()
dfUbicacion = dfUbicacion.rename_axis('Ubicacion')
dfUbicacion = dfUbicacion.reset_index(name='Numero')

dfUbicacion

,Ubicacion,Numero
0,Casa,4749
1,Recuperado,1666
2,Hospital,420
3,Fallecido,324
4,Hospital UCI,122
5,Nan,4


#Gráficas

#Miscelanea

##Pandas
[Top de funciones para aprendices](https://towardsdatascience.com/top-3-pandas-functions-for-beginners-36373ee47075)

[Funciones para usar más seguido](https://towardsdatascience.com/3-pandas-functions-you-should-be-using-more-often-696b41e8d7ca)

[Funciones tal vez desconocidas](https://towardsdatascience.com/top-3-pandas-functions-you-dont-know-about-probably-5ae9e1c964c8)